# 불법주정차 데이터 위도/경도 수집 (Kakao Api)
- https://developers.kakao.com
- 카카오맵 api 허용 설정 필수

### Present working directory

In [1]:
!pwd

/Users/hoyun/Documents/GitHub/Data_projects/dacon_sri_suwon_data_analysis


### 패키지 설치

In [2]:
!pip install pandas numpy tqdm requests

### 패키지 선언

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import json
import requests

### 카카오 API키 및 API 동작확인

In [7]:
import os
from dotenv import load_dotenv

load_dotenv()
KAKAO_API_KEY = os.getenv("KAKAO_API_KEY")
api_key = KAKAO_API_KEY

In [8]:
addr = '서울시 송파구 위례광장로 185'
url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
headers = {
    "Authorization": f"KakaoAK {api_key}"
}
params = {"query": addr}

result = json.loads(str(requests.get(url, headers=headers).text))
display(result)

{'documents': [{'address': {'address_name': '서울 송파구 장지동 878',
    'b_code': '1171010900',
    'h_code': '1171064700',
    'main_address_no': '878',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '송파구',
    'region_3depth_h_name': '위례동',
    'region_3depth_name': '장지동',
    'sub_address_no': '',
    'x': '127.14004016241',
    'y': '37.4804774688545'},
   'address_name': '서울 송파구 위례광장로 185',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 송파구 위례광장로 185',
    'building_name': '위례신도시 송파푸르지오',
    'main_building_no': '185',
    'region_1depth_name': '서울',
    'region_2depth_name': '송파구',
    'region_3depth_name': '장지동',
    'road_name': '위례광장로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '127.14004016241',
    'y': '37.4804774688545',
    'zone_no': '05848'},
   'x': '127.14004016241',
   'y': '37.4804774688545'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

### 데이터 불러오기

In [9]:
#10만개 단위(약 40만개)
data1 = pd.read_csv('parking_notcctv_1.csv')
data2 = pd.read_csv('parking_notcctv_2.csv')
data3 = pd.read_csv('parking_notcctv_3.csv')
data4 = pd.read_csv('parking_notcctv_4.csv')
data5 = pd.read_csv('parking_notcctv_5.csv')

### 데이터 수집 및 적용 함수

In [10]:
def get_lat_lng(addr):
    headers = {
        "Authorization": f"KakaoAK {api_key}"
    }
    params = {"query": addr}
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    
    response = requests.get(url, headers=headers, params=params)
    time.sleep(0.15)
    if response.status_code == 200:
        documents = response.json().get("documents")
        if documents:
            return float(documents[0]["y"]), float(documents[0]["x"])  # 위도, 경도
    return None, None


def add_to_pos(data):
    tqdm.pandas()

    latitudes = []
    longitudes = []

    for i,(_, row) in enumerate(tqdm(data.iterrows(), total=len(data))):
        if i>0 and i%1000 == 0:
            print(f'{i} rows. 중간 딜레이 5se\n')
    
        
        lat,lng = get_lat_lng(row['단속장소'])
        latitudes.append(lat)
        longitudes.append(lng)

    data['latitude'] = latitudes
    data['longitude'] = longitudes

    #display(data.head(10))
    return data

In [11]:
#샘플 테스트
sample=data1.sample(100)

### 적용

In [12]:
add_to_pos(sample)

100%|██████████| 100/100 [00:26<00:00,  3.81it/s]


,집계년도,시군명,관리기관명,단속일시정보,단속장소,단속방법,데이터기준일자,단속일시_월,단속일시_연,단속일시_일,단속일시_요일,단속일시_시,단속일시_isdaytime,단속일시_season,latitude,longitude
75821,2022,수원시,팔달구청,2022-05-25 14:15:00,고등동 고등동 고등로33번길,주행형,2023-12-01,5,2022,25,Wednesday,14,afternoon,spring,37.274043,127.001757
48075,2022,수원시,권선구청,2022-04-07 08:09:00,호매실동 금호로,주행형,2023-12-01,4,2022,7,Thursday,8,morning,spring,NaN,NaN
85872,2022,수원시,영통구청,2022-06-13 08:14:00,매탄동 매탄로126번길,주행형,2023-12-01,6,2022,13,Monday,8,morning,summer,37.261470,127.046333
58804,2022,수원시,권선구청,2022-04-25 11:15:00,경기도수원시권선구권선동 수원시 권선구 경수대로 371-1구두수선대 2019 13,보행,2023-12-01,4,2022,25,Monday,11,morning,spring,37.259704,127.023286
40417,2022,수원시,권선구청,2022-03-24 10:39:00,고색동 산업로155번길,주행형,2023-12-01,3,2022,24,Thursday,10,morning,spring,37.245967,126.985438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42995,2022,수원시,팔달구청,2022-03-29 10:33:00,경기도수원시장안구정자동 수원시 팔달구 효원로 292번길 33-22,보행,2023-12-01,3,2022,29,Tuesday,10,morning,spring,NaN,NaN
93495,2022,수원시,권선구청,2022-06-25 16:57:00,당수동 742,국민신문고,2023-12-01,6,2022,25,Saturday,16,afternoon,summer,37.292930,126.939038
58348,2022,수원시,장안구청,2022-04-24 16:16:00,정자동 장안로104번길,주행형,2023-12-01,4,2022,24,Sunday,16,afternoon,spring,37.293701,127.002320
41451,2022,수원시,팔달구청,2022-03-25 15:07:00,경기도수원시팔달구화서동 수원시 팔달구 매산로2가 3-5,보행,2023-12-01,3,2022,25,Friday,15,afternoon,spring,NaN,NaN


### 실제 데이터 주소 -> 위/경도 전환 
시간 소요될 수 있음. 요청 당 0.15초 딜레이

In [15]:
partial_data = data3

In [17]:
chunks = np.array_split(partial_data, 10)

processed_chunks = []

for i, chunk in enumerate(chunks):
    print(f"Processing chunk {i+1}/10")
    result = add_to_pos(chunk)

    result.to_csv(f'processed_chunk_data3 {i+1}.csv', index=False)
    processed_chunks.append(result)

/opt/anaconda3/envs/data_venv/lib/python3.10/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Processing chunk 1/10


  0%|          | 0/10000 [00:00<?, ?it/s]

 10%|█         | 1000/10000 [04:04<38:09,  3.93it/s] 

1000 rows. 중간 딜레이 5se



...

 80%|████████  | 8000/10000 [33:48<08:07,  4.10it/s]

8000 rows. 중간 딜레이 5se



 90%|█████████ | 9000/10000 [37:55<03:56,  4.22it/s]

9000 rows. 중간 딜레이 5se



100%|██████████| 10000/10000 [42:02<00:00,  3.96it/s]


In [18]:
final_data = pd.concat(processed_chunks, ignore_index=True)
final_data.to_csv('data3_processed_data.csv', index=False)